In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup
import sqlite3
import time

In [21]:
driver = webdriver.Chrome('C:/PyApp/driver/chromedriver')
driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
time.sleep(5)

In [22]:
driver.find_element_by_xpath("""//*[@id="id"]""").send_keys('id') # id를 입력한다. 
driver.find_element_by_xpath("""//*[@id="inputPwd"]""").send_keys('pw') # 패스워드를 입력한다. 
driver.find_element_by_xpath("""//*[@id="loginBtn"]""").click() # 입력 버튼 클릭.

time.sleep(5)

In [23]:
CAFE_NAME = 'ssaumjil' 
BOARD_NAME = 'EnXG' 
DB = CAFE_NAME + '_m_article.db'

In [24]:
conn = sqlite3.connect(DB)
cur = conn.cursor()

sql = 'CREATE TABLE IF NOT EXISTS "' + BOARD_NAME + \
  '"("num" INTEGER NOT NULL UNIQUE, "subject" TEXT, ' + \
  '"nick" TEXT, "write_time" TEXT, "views" INTEGER, ' + \
  '"url" TEXT, "contents" TEXT, PRIMARY KEY("num"));'

cur.execute(sql)
conn.commit()

sql = 'CREATE TABLE IF NOT EXISTS "' + BOARD_NAME + \
  '_cmt" ("cmt_num" TEXT NOT NULL UNIQUE, "cmt_reply" INTEGER, "cmt_writer" TEXT, "cmt_time" TEXT, "cmt_txt" TEXT, PRIMARY KEY("cmt_num"));'

cur.execute(sql)
conn.commit()

conn.close()

In [ ]:
inp_num = input('저장할 게시물 번호 + 엔터: ')

num = int(inp_num)

url = 'http://m.cafe.daum.net/%s/%s/%s' % (CAFE_NAME, BOARD_NAME, inp_num)
driver.get(url)

time.sleep(5)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser') 

subject = soup.body.find('h3', class_='tit_subject')

if subject is None:
    print(url, '지워진 게시물입니다.')
else:
    subject = subject.get_text(strip=True)  
    
    if soup.body.find('span', class_='txt_subject').find('span', class_='sr_only').get_text() == '작성자':
        nick = soup.body.find('span', class_='sr_only').next_sibling
    else:
        nick = ''
    
    num_subject = soup.body.find_all('span', class_='num_subject')
    write_time = num_subject[0].get_text()
    views = num_subject[1].get_text()

    print(num, subject, nick, write_time, views, url)

    contents = soup.body.find('div', id='article').get_text('\n', strip=True)
    print(contents)
   
    conn = sqlite3.connect(DB)
    cur = conn.cursor()
    sql = "replace into %s(num,subject,nick,write_time,views,url,contents) values (?,?,?,?,?,?,?)" % BOARD_NAME
    cur.execute(sql, (num, subject, nick, write_time, views, url, contents))
    conn.commit()
    conn.close()
    
    cmt_total = soup.body.find('article', id='mArticle').find('span', class_="num_total").get_text()
    print('댓글 개수 :', cmt_total)

    driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments'.format(CAFE_NAME,BOARD_NAME,inp_num))
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    cmt_page_max = int(soup.body.find('span', id="pagingNav").find_all('span', class_="num_page").pop().get_text())

    print('cmt_page_max', cmt_page_max)
    for i in range(cmt_page_max):
        j = cmt_page_max-i
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        cmt_all = soup.body.find('ul' ,id="commentList").find_all('li')
        
        for k in cmt_all:
            if k.div.span.find_all('span', class_="txt_bar"):
                if k['class'] == ['reply_on']:
                    cmt_reply = True
                    print('ㄴ', end='')
                else:
                    cmt_reply = False

                cmt_num = inp_num +'_'+ k['id'].split('comment_')[1]
                cmt_writer = k.div.find('span', class_='sr_only').next_sibling
                cmt_time = k.div.span.find('span', class_="num_info").get_text()
                cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                print(cmt_num, '[', cmt_writer, ']', cmt_time, cmt_txt)
                
                conn = sqlite3.connect(DB)
                cur = conn.cursor()
                sql = "replace into '%s_cmt'(cmt_num,cmt_reply,cmt_writer,cmt_time,cmt_txt) values (?,?,?,?,?)" % BOARD_NAME
                cur.execute(sql, (cmt_num,cmt_reply,cmt_writer,cmt_time,cmt_txt))
                conn.commit()
                conn.close()
                
        if j > 1:
            driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments?prev_page={3}&mode=regular&cdepth={4}&page={5}'.format(CAFE_NAME,BOARD_NAME,inp_num,j,'0002100000',(j-1)))
            time.sleep(2)

In [101]:
driver.close()